In [ ]:
# Importing the Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')
df.head()

In [ ]:
# Shape of the Dataset
print ("The shape of the dataset : ", df.shape)

In [ ]:
df.dtypes

In [ ]:
# Filling the Null Values
df['adults'].fillna(0, inplace = True)
df['children'].fillna(0, inplace = True)
df['babies'].fillna(0, inplace = True)

df['children'] = df['children'].astype(int)

## Exploratory Data Analysis and SA

#### Distribution of the hotel based on the booking status

In [ ]:
# Parameters
plt.rcParams['figure.figsize'] = [10, 7]
sns.set(style = 'white', font_scale = 1.3)

#1: canceled and 0: not canceled
df['is_canceled'].replace({1: 'canceled', 0: 'not canceled'}, inplace = True)

# Plot
dist = sns.countplot(df['hotel'], hue = 'is_canceled', data = df, palette = 'Set2');

dist.set(title = "Distribution of the hotel based on the booking status");

#### Number of Reservations per year (months)

In [ ]:
# parameters
plt.rcParams['figure.figsize'] = [20, 7]
sns.set(style = 'white', font_scale = 1.25)

# plot
arrival = sns.countplot(df['arrival_date_year'], hue = 'arrival_date_month', data = df, palette = 'Set3',
             hue_order = ['January', 'February', 'March', 'April', 'May', 'June',
                          'July', 'August', 'September', 'October', 'November', 'December'])
arrival.set(title = "Number of Reservations per year (months)")
plt.legend(loc='upper left');

#### Reservation Status per year

In [ ]:
# Parameters
plt.rcParams['figure.figsize'] = [15, 7]
sns.set(style = 'darkgrid', font_scale = 1.2)

# catplot (countplot)
cat = sns.catplot(x = 'reservation_status', col = 'arrival_date_year', kind = 'count', data = df,
            height = 6, palette = ['orange', 'mediumaquamarine', 'darkgrey']);

In [ ]:
fig, ax = plt.subplots(1, 3)
sns.set(font_scale = 1.2)

val_2015 = df['reservation_status'].groupby(df['arrival_date_year']).value_counts()[2015].values
val_2016 = df['reservation_status'].groupby(df['arrival_date_year']).value_counts()[2016].values
val_2017 = df['reservation_status'].groupby(df['arrival_date_year']).value_counts()[2017].values
index = list(df['reservation_status'].groupby(df['arrival_date_year']).value_counts()[2015].index)

color = ['orange', 'mediumaquamarine', 'darkgrey']

# 2015
ax[0].pie(val_2015, labels = index, autopct='%.02f%%', explode = [0, 0, 0.05], colors = color)
ax[0].set_title('2015')

# 2016
ax[1].pie(val_2016, labels = index, autopct='%.02f%%', explode = [0, 0, 0.05], colors = color)
ax[1].set_title('2016')

# 2017
ax[2].pie(val_2017, labels = index, autopct='%.02f%%', explode = [0, 0, 0.05], colors = color);
ax[2].set_title('2017')
plt.show()

#### Donut Graph showing the canceled and non canceled booking each year

In [ ]:
# Donut Graph showing the canceled and non canceled reservations each year
val_2015 = df['is_canceled'].groupby(df['arrival_date_year']).value_counts()[2015].values
val_2016 = df['is_canceled'].groupby(df['arrival_date_year']).value_counts()[2016].values
val_2017 = df['is_canceled'].groupby(df['arrival_date_year']).value_counts()[2017].values

t = np.array([val_2015, val_2016, val_2017])

fig, ax = plt.subplots()
sns.set(font_scale = 1.3)
size = 0.3 

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
inner_colors = cmap(np.array([1, 2, 5, 6, 9, 10]))

ax.pie(t.sum(axis = 1), radius=1, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%.01f%%')

ax.pie(t.flatten(), radius=1-size, colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'), labels = ['nc', 'c', 'nc', 'c', 'nc', 'c'])

ax.set(aspect="equal", title='Canceled and Non canceled reservations each year')
ax.legend([2015, 2016, 2017], loc = 'best');

#### Canceled and Non Canceled Booking according to the Dates

In [ ]:
# Concatenation of arrivale date from year, month and day
df['arrival_date_month_num'] = df['arrival_date_month'].apply(lambda x: datetime.datetime.strptime(x, '%B').month)

df_1 = pd.DataFrame()

# converting to datetime
df_1[['year', 'month', 'day']] = df[["arrival_date_year", "arrival_date_month_num", "arrival_date_day_of_month"]].copy()
df['arrival_date'] = pd.to_datetime(df_1[['year', 'month', 'day']])

# Sorting the dataframe by arrival dates
df.sort_values(by = ['arrival_date'], inplace = True)
del df_1

df['arrival_date'][0:10]

In [ ]:
# Grouping the arrival dates and counting the canceled and not canceled reservations
df.replace({'not canceled' : -1, 'canceled' : 1}, inplace = True)
a = df[['is_canceled', 'arrival_date']].groupby(df['arrival_date'])['is_canceled'].value_counts()

# timestamps
b = [a.index[x][0] for x in range(len(a))]

# canceled or not canceled array
c = [a.index[x][1] for x in range(len(a))]

# 1 represents the canceled and -1 represents not canceled
g = a.values * c # below zero axis it will be not canceled and above it will be canceled


fig = px.bar(x = b, y = g, title = 'Canceled and Non Canceled Reservation according to the Dates',
            color = g,
            labels = {
                     "x": "Date",
                     "y": "Reservations (c or nc)",
                 })
fig.show()

#### Hotel Choices for Adults without children, Adults with children and Only Children

In [ ]:
# These are the datapoints where there is no adults, children and babies so we will not consider these points

_ = df[(df.children ==0)&(df.babies ==0)&(df.adults == 0)].shape
print ("Data Points without any adults, children and babies : ", _)

# Dropping those data points
df_1 = df[(df.children !=0) | (df.babies !=0) | (df.adults != 0)]

only_adults = df_1[(df_1.children ==0)&(df_1.babies ==0)]
adults_w_child = df_1[(df_1.children !=0) | (df_1.babies !=0)]
only_childs = df_1[df_1.adults == 0]

print ("Only Adults : ", only_adults.shape[0])
print ("Adults with children : ", adults_w_child.shape[0])
print ("Only children and babies : ", only_childs.shape[0])

In [ ]:
# Hotels
hotel = ['City Hotel', 'Resort Hotel']

fig = go.Figure()

# Adults w/o children
fig.add_trace(go.Bar(
    x = hotel,
    y = only_adults['hotel'].value_counts().tolist(),
    name='Adults w/o children',
    marker_color='indianred'
))

# Adults w/ children
fig.add_trace(go.Bar(
    x = hotel,
    y = adults_w_child['hotel'].value_counts().tolist(),
    name='Adults w/ children',
    marker_color='lightsalmon'
))

# Only children
fig.add_trace(go.Bar(
    x = hotel,
    y = only_childs['hotel'].value_counts().tolist(),
    name='Only children'
))

# Here we modify the layout of the figure
fig.update_layout(
    barmode='group',
    xaxis_tickangle=0,
    autosize=False,
    width = 750,
    height = 900
)
fig.show()

#### Total Number of guest from different countries whose bookings are not canceled

In [ ]:
# total people who booked from the countries
df['total_guest'] = df['adults'] + df['children'] + df['babies']

# non canceled bookings
df.replace({-1 : 0, 1 : 1}, inplace = True)
df1 = df.loc[df.is_canceled == 0, :].copy()

countries = pd.DataFrame(df1[['total_guest', 'country']].groupby(df1['country'])['total_guest'].sum())

print ("Total Countries in the dataset : ", df['country'].nunique())

# piechart
fig = px.pie(countries, values = 'total_guest', names = countries.index, title = 'Total Number of guest from different countries whose bookings are not canceled')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

#### Total Number of Guests from Different Countries on World Map

In [ ]:
# show on map
world_map = px.choropleth(countries,
                    locations = countries.index,
                    color = 'total_guest', 
                    hover_name = countries.index, 
                    color_continuous_scale = px.colors.sequential.Sunsetdark,
                    title = "Total Number of Guests from Different Countries on World Map")

world_map.update_geos(fitbounds = "locations", visible = False)
# world_map.update_layout(margin = {"r":0,"t":0,"l":0,"b":0})

world_map.show()

#### Total Number of Hotel Guests per month

In [ ]:
# Non Canceled bookings
df_1 = df.loc[df.is_canceled == 0, :].copy()

# Total Guest for Resort Hotel
a = df_1.loc[df.hotel == 'Resort Hotel', :]

b = a[['arrival_date_month', 'total_guest']].groupby(a['arrival_date_month']).sum()
b['hotel'] = ['Resort Hotel']*len(b)
c = b.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

# Total guest for City Hotel
a1 = df_1.loc[df.hotel == 'City Hotel', :]

b1 = a1[['arrival_date_month', 'total_guest']].groupby(a1['arrival_date_month']).sum()
b1['hotel'] = ['City Hotel']*len(b1)
c1 = b1.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

hot = pd.concat([c, c1])
hot['month'] = hot.index

plt.rcParams['figure.figsize'] = [12, 8]
sns.set(style = 'whitegrid', font_scale = 1.3)
ax = sns.lineplot(x = 'month', y="total_guest", data=hot, hue = 'hotel', sort = False, style = 'hotel', markers=True, dashes = False)
ax.set(title = "Total Number of Hotel Guest per month")
plt.setp(ax.get_xticklabels(), rotation = 45);

We can see the maximum number of hotel guests are there in August followed by July for both types of hotel. This can occur also because the data given for August and July span over 3 years and for all the other months the data is from 2 years. This discrepancy can be corrected if we take the average number of guests. The minimum number of guest is in January.

#### Average number of guest per month  (Busiest Month over the years)

In [ ]:
# preproc
df_1 = df.loc[df.is_canceled == 0, :].copy()
a = df_1[['hotel', 'arrival_date_month']].copy()

rh = pd.DataFrame(a.loc[a.hotel == 'Resort Hotel', 'arrival_date_month'].value_counts())
rh['hotel'] = ['Resort Hotel']*len(rh)
rh = rh.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

ch = pd.DataFrame(a.loc[a.hotel == 'City Hotel', 'arrival_date_month'].value_counts())
ch['hotel'] = ['City Hotel']*len(ch)
ch = ch.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

fin = pd.concat([rh, ch])
fin['months'] = fin.index

# Finding the average number of guest per month over the year the data is available
def average(x, y):
    if y not in ['July', 'August']:
        return x/2
    else:
        return x/3

fin['avg'] = fin.apply(lambda x: average(x.arrival_date_month, x.months), axis = 1)

ax1 = sns.lineplot(fin['months'], fin['avg'], hue = 'hotel', data = fin, sort = False, lw = 2)
ax1.set(title = "Average number of guest per month", ylabel = "Average number of Guest")
plt.setp(ax1.get_xticklabels(), rotation = 45);

#### Rooms Assigned to the Guests according to their reservation and the Reservation Status of the guest who are assigned different rooms

In [ ]:
# Parameters
plt.rcParams['figure.figsize'] = [20, 9]
fig, ax = plt.subplots(1, 2)
sns.set(font_scale = 1.1)

r = df[['is_canceled', 'reserved_room_type', 'assigned_room_type']].copy()

def samediff(x, y):
    if x == y:
        return ("Same")
    else:
        return ("Diff")
    
r['assignment'] = r.apply(lambda x : samediff(x.reserved_room_type, x.assigned_room_type), axis = 1)

len(r.loc[r.assignment == 'Diff', :]['is_canceled'])
cnc = r.loc[r.assignment == 'Diff', :]['is_canceled'].value_counts()

r = r['assignment'].value_counts()

# pie chart for the assigned room type
ax[0].pie(r.values, labels = ['Same room type assigned', 'Different room type assigned'],
        autopct='%.02f%%', explode = [0, 0.05], colors = ['mediumorchid', 'mediumturquoise']);
ax[0].set_title('Room Assigned to the Guest according to their reservation');

# pie chart for the reservations status who are assigned different rooms
ax[1].pie(cnc.values, labels = ['Not Cancelled', 'Cancelled'],
        autopct='%.02f%%', explode = [0, 0.05], colors = ['seagreen', 'firebrick']);
ax[1].set_title('Reservation Status of the guest who are assigned different rooms');

#### Average Daily Rate of the Both Type of Hotels

ADR - The average daily rate is calculated by taking the average revenue earned from rooms and dividing it by the number of rooms sold. It excludes complimentary rooms and rooms occupied by staff.

In [ ]:
# Removing the ghost values
df = df.loc[(df.children != 0)|(df.babies != 0)|(df.adults != 0), :]

# Rate per person
df['adr_per_person'] = df['adr']/(df['adults'] + df['children'])

In [ ]:
# Parameters
plt.rcParams['figure.figsize'] = [25, 9]
fig, ax = plt.subplots(1, 2)
sns.set(style = 'whitegrid', font_scale = 1.3)

nc = df.loc[df.is_canceled == 0, :].copy()
rh = nc.loc[nc.hotel == 'Resort Hotel', ['adr_per_person', 'assigned_room_type', 'hotel']]
ch = nc.loc[nc.hotel == 'City Hotel', ['adr_per_person', 'assigned_room_type', 'hotel']]

# Final df
fin = pd.concat([rh, ch], ignore_index = True)

# Boxplot Showing the 5 number summaries of the room price per person per night for the different room
box = sns.boxplot(y = fin.adr_per_person, data = fin, x = fin.assigned_room_type, hue = 'hotel', fliersize=0, ax = ax[0])
box.set(title = 'Price of the room per night per person (Boxplot)', xlabel = 'Room Type', ylabel = 'Price', ylim = (0, 200))
box.legend(loc='upper left');

# lineplot showing the variation of the room price per person per night with 95% confidence interval
line = sns.lineplot(y = fin.adr_per_person, data = fin, x = fin.assigned_room_type, hue = 'hotel', ci = 95, ax = ax[1])
line.set(title = 'Price of the room per night per person (Lineplot)', xlabel = 'Room Type', ylabel = 'Price', ylim = (0, 80));

In [ ]:
resort_mean = nc.loc[(nc.is_canceled == 0) & (nc.hotel == 'Resort Hotel'), 'adr_per_person'].mean()
city_mean = nc.loc[(nc.is_canceled == 0) & (nc.hotel == 'City Hotel'), 'adr_per_person'].mean()

print ("The mean price of the rooms per night per person for Resort Hotel is {:.2f}".format(resort_mean))
print ("The mean price of the rooms per night per person for City Hotel is {:.2f}".format(city_mean))

#### Reservation status of the Guests with and without special requests

In [ ]:
# Parameters
plt.rcParams['figure.figsize'] = [20, 10]
fig, ax = plt.subplots(1, 2)
sns.set(style = 'darkgrid', font_scale = 1.2)

# Special and non Special Request of the guest
special_req = df[df.total_of_special_requests != 0].copy()
non_special_req = df[df.total_of_special_requests == 0].copy()

# Plots
spec = sns.countplot(special_req.reservation_status, ax = ax[0], palette = 'winter')
spec.set(title = 'Guest with special requests', xlabel = 'Reservation_status')

n_spec = sns.countplot(non_special_req.reservation_status, ax = ax[1], palette = 'winter')
n_spec.set(title = 'Guest without special requests', ylabel = "", xlabel = 'Reservation_status');

#### Average Number of Special request over the months [95% ci]

In [ ]:
sns.set(style = 'whitegrid', font_scale = 1.2)

a = sns.lineplot(y = df.total_of_special_requests, x = df.arrival_date_month_num, data = df, color = 'darkcyan')
plt.xticks(np.arange(1, 13, 1))
a.set_xticklabels(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

a.set(title = 'Average Number of Special request over the months [95% ci]', xlabel = 'Month', ylabel = 'Special request');

#### Spearman and Pearson correlation heatmap

In [ ]:
# Pearson Correlation Heatmap
plt.rcParams['figure.figsize'] == [18, 16]
sns.set(font_scale = 0.8)

sns.heatmap(df.corr(method = 'pearson'), annot = True, cmap = "YlGnBu");

In [ ]:
# Spearman Correlaton Heatmap
sns.heatmap(df.corr(method = 'spearman'), annot = True, cmap = "viridis");

## Preprocessing

In [ ]:
# Unique Object values
a = df.select_dtypes(object).columns
for i in a:
    print (i, df[i].nunique())

In [ ]:
# columns with dtype object
categorical_features = list(df.select_dtypes(include=['object']).columns)
categorical_features.remove('reservation_status_date')

# Label Encoder 
label_encoder_feat = {}
for i, feature in enumerate(categorical_features):
    df[feature] = df[feature].astype(str)
    label_encoder_feat[feature] = LabelEncoder()
    df[feature] = label_encoder_feat[feature].fit_transform(df[feature])

In [ ]:
# Filling agent null values 
df.agent.fillna(0, inplace = True)
df.agent = df.agent.astype(int)

In [ ]:
# Checking the correlation to choose the features to use in the Modelling
# Choosing the threshold Correlation as 0.04 (except for children)

df.corr(method = 'pearson')['is_canceled']

In [ ]:
# The features that are going to be used for the training and testing

df_hotel = df[['hotel', 'lead_time', 'adults', 'children', 'babies', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest',
   'previous_cancellations', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
   'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests', 'is_canceled', 'adr', 'total_guest']].copy()

df_hotel.head()

#### Cross Validation - StratifiedKFolds

In [ ]:
df_hotel["kfold"] = -1

# shuffling
df_hotel = df_hotel.sample(frac = 1).reset_index(drop = True)

# targets
target = df_hotel.is_canceled.values

# stratified k fold with 5 folds
kf = model_selection.StratifiedKFold(n_splits = 5)

for i, (train, val) in enumerate(kf.split(X = df_hotel, y = target)):
    df_hotel.loc[val, 'kfold'] = i

In [ ]:
df_hotel.kfold.unique()

In [ ]:
# This fuction will run 5 folds for each model and the performance can be seen during training.

def run(folds, model):

    # training and validation set
    df_train = df_hotel[df_hotel.kfold != folds].reset_index(drop = True)
    df_valid = df_hotel[df_hotel.kfold == folds].reset_index(drop = True)

    X_train = df_train.drop(columns = ['is_canceled']).values
    y_train = df_train.is_canceled.values

    X_valid = df_valid.drop(columns = ['is_canceled']).values
    y_valid = df_valid.is_canceled.values

    clf = models[model]

    print ("Training...")
    clf.fit(X_train, y_train)

    print ("Done!!")
    preds = clf.predict(X_valid)
        
    acc = metrics.accuracy_score(y_valid, preds)
    print ("Fold = {} Accuracy = {}".format(folds, acc))
    print ("------Classification Report")
    print (metrics.classification_report(y_valid, preds))

## Modelling

In [ ]:
# Models Trained + DNN
models = {
    'logistic_regression' : LogisticRegression(C = 1.2),
    'xgboost': XGBClassifier(eta = 0.35, max_depth = 12),
    'random_forest' :  RandomForestClassifier(n_estimators = 150)
    }

#### Logistic Regression

In [ ]:
# 5 Folds -- Logistic Regression
for fold in range(0, 5):
    run(fold, 'logistic_regression')

#### XGBoost

In [ ]:
# 5 Folds -- XGBoost
for fold in range(0, 5):
    model = run(fold, 'xgboost')

#### Random Forest

In [ ]:
# 5 Folds -- Random Forest
for fold in range(0, 5):
    run(fold, 'random_forest')

#### Deep Neural Network

In [ ]:
# Model
def dnn(input_shape):
    dnn = keras.Sequential([
        layers.BatchNormalization(input_shape = [input_shape]),
        layers.Dense(512, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(512, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(256, activation = 'relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation = 'sigmoid')
    ])

    dnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
    )
    return dnn

In [ ]:
# 5 folds
for fold in range(0, 5):
    df_train = df_hotel[df_hotel.kfold != fold].reset_index(drop = True)
    df_valid = df_hotel[df_hotel.kfold == fold].reset_index(drop = True)

    X_train = df_train.drop(columns = ['is_canceled']).values
    y_train = df_train.is_canceled.values

    X_valid = df_valid.drop(columns = ['is_canceled']).values
    y_valid = df_valid.is_canceled.values
    
    clf = dnn(X_train.shape[1])
    
    if fold == 0:
        clf.summary()
    
    # Training
    print ("Training...")
    clf.fit(
        X_train, y_train,
        batch_size = 512,
        epochs = 50,
        verbose = 0
    )
    
    print ("Done!!")
    
    preds = clf.predict(X_valid)
    # default threshold 0.5
    preds = (preds>0.5).astype(int)
    
    acc = metrics.accuracy_score(y_valid, preds)
    print ("Fold = {} Accuracy = {}".format(fold, acc))
    print ("------Classification Report------")
    print (metrics.classification_report(y_valid, preds))

[GitHub Link](https://github.com/SumitM0432/EDA-SA-and-Hotel-Booking-Cancellation-Prediction.git)